In [2]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
from matplotlib.colors import ListedColormap

import re
import neurokit2 as nk
import sys

In [3]:
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.8
plt.rcParams['grid.linestyle'] = 'dotted'
plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams['figure.figsize'] = (4.845, 3.135)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['mathtext.default']='regular'

In [4]:
sys.path.append(os.path.relpath("../src/"))
from dataloader import S1, S2, S3, S4

s1 = S1()
# s2 = S2()
# s3 = S3()
# s4 = S4()

In [40]:
# generate the parameters for epochs creating
def epochs_info_generator(data, window_size, step):
    events_onset = []
    epochs_start = []
    epochs_end = []
    cur_index = 0
    while(cur_index + window_size <= len(data)):
        events_onset.append(cur_index)
        epochs_start.append(0)
        epochs_end.append(window_size / 1000)
        cur_index += step
    if cur_index != len(data) and cur_index + window_size > len(data):
        events_onset.append(cur_index)
        epochs_start.append(0)
        epochs_end.append((len(data) - cur_index) / 1000)
        
    return events_onset, epochs_start, epochs_end

In [70]:
def process_feature_extraction(scenario: int, sub, vid, sampling_rate, window_size, step):
    data_loader = DataLoader(scenario)
    s1.train_data(sub, vid)
    train_data = data_loader.train_data(sub, vid)
    annotations = train_data[1]
    
    # signal processing
    ecg_signals, _ = nk.ecg_process(train_data[0]['ecg'], sampling_rate=sampling_rate)
    ppg_signals, _ = nk.ppg_process(train_data[0]['bvp'], sampling_rate=sampling_rate)
    eda_signals, _ = nk.eda_process(train_data[0]['gsr'], sampling_rate=sampling_rate)
    rsp_signals, _ = nk.rsp_process(train_data[0]['rsp'], sampling_rate=sampling_rate)
    emg_zygo_signals, _ = nk.emg_process(train_data[0]['emg_zygo'], sampling_rate=sampling_rate)
    emg_coru_signals, _ = nk.emg_process(train_data[0]['emg_coru'], sampling_rate=sampling_rate)
    emg_trap_signals, _ = nk.emg_process(train_data[0]['emg_trap'], sampling_rate=sampling_rate)
    
    physiological_signals = [ecg_signals, ppg_signals, eda_signals, rsp_signals, emg_zygo_signals, emg_coru_signals, emg_trap_signals]
    # physiological_signals = [ecg_signals, ppg_signals, eda_signals, rsp_signals]
    physiological_features = None
    for i, signals in enumerate(physiological_signals):
        events, epochs_start, epochs_end = epochs_info_generator(signals, window_size=window_size, step=step)
        windows = nk.epochs_create(signals, events, sampling_rate=sampling_rate, epochs_start=epochs_start, epochs_end=epochs_end)
        
        if i > 3:
            features = nk.emg_analyze(windows, sampling_rate=sampling_rate, method="event-related")
        elif i == 0:
        # if i == 0:
            features = nk.ecg_analyze(windows, sampling_rate=sampling_rate, method="event-related")
        elif i == 1:
            features = nk.ppg_analyze(windows, sampling_rate=sampling_rate, method="event-related")
        elif i == 2:
            features = nk.eda_analyze(windows, sampling_rate=sampling_rate, method="event-related")
        else:
            features = nk.rsp_analyze(windows, sampling_rate=sampling_rate, method="event-related")
            
        
        if physiological_features is None:
            physiological_features = features
        else:
            # physiological_features = pd.concat([physiological_features, features.reindex(physiological_features)], axis=1)
            physiological_features = pd.concat([physiological_features, features.drop(['Label', 'Event_Onset'], axis=1)], axis=1)
            
    return physiological_features

In [71]:
physiological_features = process_feature_extraction(scenario=1, sub=4, vid=1, sampling_rate=1000, window_size=4000, step=50) 

/home/celia/.local/lib/python3.10/site-packages/neurokit2/events/events_find.py:116: NeuroKitWarning: No events found. Check your event_channel or adjust 'threshold' or 'keep' arguments.
  warn(
/home/celia/.local/lib/python3.10/site-packages/neurokit2/events/events_find.py:116: NeuroKitWarning: No events found. Check your event_channel or adjust 'threshold' or 'keep' arguments.
  warn(
/home/celia/.local/lib/python3.10/site-packages/neurokit2/events/events_find.py:116: NeuroKitWarning: No events found. Check your event_channel or adjust 'threshold' or 'keep' arguments.
  warn(


In [72]:
physiological_features

,Label,Event_Onset,ECG_Rate_Baseline,ECG_Rate_Max,ECG_Rate_Min,ECG_Rate_Mean,ECG_Rate_SD,ECG_Rate_Max_Time,ECG_Rate_Min_Time,ECG_Rate_Trend_Linear,...,EMG_Amplitude_Max,EMG_Amplitude_SD,EMG_Amplitude_Max_Time,EMG_Bursts,EMG_Activation,EMG_Amplitude_Mean,EMG_Amplitude_Max,EMG_Amplitude_SD,EMG_Amplitude_Max_Time,EMG_Bursts
1,1,0,79.072375,11.699183,-0.538343,2.975773,3.996458,3.700925,1.590398,-2.772489,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2,50,79.072375,11.699183,-0.538343,3.101940,4.059328,3.650913,1.540385,-2.481018,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,3,100,79.072375,11.699183,-0.538343,3.223442,4.109801,3.600900,1.490373,-2.140155,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4,4,150,79.072375,11.699183,-0.538343,3.340440,4.149389,3.550888,1.440360,-1.754987,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
5,5,200,79.072375,11.699183,-0.538343,3.453284,4.179636,3.500875,1.390348,-1.331553,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,1978,98850,73.534469,5.102491,-0.274395,2.018257,1.808270,2.086522,0.508127,3.181611,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1979,1979,98900,73.487225,5.149734,-0.227152,2.108174,1.799813,2.036509,0.458115,3.279028,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1980,1980,98950,73.443604,5.193355,-0.183531,2.195036,1.789618,1.986497,0.408102,3.364228,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1981,1981,99000,73.403899,5.233061,-0.143826,2.278503,1.777657,1.936484,0.358090,3.436700,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
